In [4]:
import json
import pandas as pd
import requests
from sqlalchemy import create_engine
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\Get_Data_Functions.py

In [6]:
def list_db_tables(db_name):
    Conn = db.connect(db_name)
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor = Conn.cursor()
    cursor.execute(sql_query)
    print(cursor.fetchall())
    Conn.close() 

In [7]:
list_db_tables('MacroData.db')

[]


In [8]:
df_data, df_dict  = create_dataframes("CUSR0000SA0", 1950, 2024)
df_data

,value,month,pct_changeMoM,pct_changeYoY,pct_changeMoM_Ann
239,23.51,1950-01,NaN,NaN,NaN
238,23.61,1950-02,0.425351,NaN,5.225330
237,23.64,1950-03,0.127065,NaN,1.535479
236,23.65,1950-04,0.042301,NaN,0.508797
235,23.77,1950-05,0.507400,NaN,6.261622
...,...,...,...,...,...
4,312.230,2024-03,0.378069,3.475131,4.632370
3,313.207,2024-04,0.312910,3.357731,3.820226
2,313.225,2024-05,0.005747,3.250210,0.068986
1,313.049,2024-06,-0.056190,2.975629,-0.672196


In [9]:
df_dict

,series_title,seasonality,survey_long_name,survey_short_name,survey_abbreviation,measure_data_type,area,item
series_id,,,,,,,,
CUSR0000SA0,"All items in U.S. city average, all urban cons...",Seasonally Adjusted,Consumer Price Index for All Urban Consumers (...,CPI-U,CU,All items,U.S. city average,All items


In [10]:
df_data.to_csv('CPI.csv', index=False)

In [11]:
conn = db.connect('MacroData.db')
df_data.to_sql(df_dict.index[0] + '_data', conn, if_exists='replace')

895

In [5]:
# create connection to the database
connection = db.connect('MacroData.db') 
  
# drop table 
connection.execute("DROP TABLE CUSR0000SA0_data") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

OperationalError: no such table: CUSR0000SA0_data